In [1]:
#importing the libraries
import time
import matplotlib.pyplot as plt
from google import genai
import pandas as pd
import requests
from tqdm import tqdm
from google.colab import userdata

In [2]:
#getting the dataframe
df=pd.read_csv('NPI_Extract_corr.csv')
# Clean column names
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
print("CSV loaded. Shape:", df.shape)
print("Columns:", df.columns.tolist())
print("\nFirst few rows:")
print(df.head())

CSV loaded. Shape: (1000, 6)
Columns: ['npi', 'provider_organization_name_(legal_business_name)', 'provider_last_name_(legal_name)', 'provider_first_name', 'provider_middle_name', 'provider_first_line_business_practice_location_address']

First few rows:
          npi provider_organization_name_(legal_business_name)  \
0  1679576722                                              NaN   
1  1588667638                                              NaN   
2  1497758544           CUMBERLAND COUNTY HOSPITAL SYSTEM, INC   
3  1306849450                                              NaN   
4  1215930367                                              NaN   

  provider_last_name_(legal_name) provider_first_name provider_middle_name  \
0                           WIEBE               DAVID                    A   
1                         PILCHER             WILLIAM                    C   
2                             NaN                 NaN                  NaN   
3                             NaN   

In [3]:
#getting the gemini api key
k=userdata.get('API_key')
client = genai.Client(api_key=k)

In [4]:
#defining the data validation agent and the npi validation agent
import requests
def data_validation_agent(row):
    """Basic format/structure validation"""
    npi = str(row.get("npi", "")).strip()
    addr = str(row.get("provider_first_line_business_practice_location_address", "")).strip()

    return {
        "npi_format_valid": len(npi) == 10 and npi.isdigit(),
        "address_present": len(addr) > 5,
        "raw_data_quality": "GOOD" if len(npi) > 0 and len(addr) > 0 else "POOR"}
def npi_validation_agent(row):
    npi = str(row.get("npi", "")).strip()
    # Basic NPI format check
    if len(npi) != 10 or not npi.isdigit():
        return {
            "npi_valid": False,
            "npi_status": "INVALID_FORMAT",
            "npi_name": "",
            "npi_address": "",
            "npi_taxonomy_code": "",
            "npi_taxonomy_desc": "",
            "name_match": False,
            "address_match": False,
        }

    url = f"https://npiregistry.cms.hhs.gov/api/?number={npi}&version=2.1"
    try:
        resp = requests.get(url, timeout=10)
        data = resp.json()

        if data.get("result_count", 0) != 1:
            return {
                "npi_valid": False,
                "npi_status": "NOT_FOUND",
                "npi_name": "",
                "npi_address": "",
                "npi_taxonomy_code": "",
                "npi_taxonomy_desc": "",
                "name_match": False,
                "address_match": False,
            }
        rec = data["results"][0]
        npi_basic = rec.get("basic", {})
        npi_addresses = rec.get("addresses", [])
        taxonomies = rec.get("taxonomies", [])
        npi_first = npi_basic.get("first_name", "") or ""
        npi_last  = npi_basic.get("last_name", "") or ""
        npi_org   = npi_basic.get("legal_business_name", "") or ""
        npi_full_indiv = f"{npi_first} {npi_last}".strip()
        npi_name = npi_org if npi_org else npi_full_indiv
        npi_address = npi_addresses[0].get("address_1", "") if npi_addresses else ""

        primary_tax = taxonomies[0] if taxonomies else {}
        npi_taxonomy_code = primary_tax.get("code", "")
        npi_taxonomy_desc = primary_tax.get("desc", "")
        csv_first = str(row.get("provider_first_name", "")).strip()
        csv_last  = str(row.get("provider_last_name_(legal_name)", "")).strip()
        csv_org   = str(row.get("provider_organization_name_(legal_business_name)", "")).strip()
        csv_addr  = str(row.get("provider_first_line_business_practice_location_address", "")).strip()
        def norm_name(s):
            return " ".join(str(s).upper().split())

        def norm_addr(s):
            return " ".join(str(s).upper().split())

        csv_full_indiv = norm_name(f"{csv_first} {csv_last}")
        npi_full_indiv = norm_name(npi_full_indiv)
        csv_full_org   = norm_name(csv_org)
        npi_full_org   = norm_name(npi_org)

        indiv_name_match = (
            csv_first != "" and csv_last != "" and
            csv_full_indiv != "" and npi_full_indiv != "" and (
                csv_full_indiv == npi_full_indiv or
                csv_full_indiv in npi_full_indiv or
                npi_full_indiv in csv_full_indiv
            )
        )

        org_name_match = (
            csv_full_org != "" and npi_full_org != "" and (
                csv_full_org == npi_full_org or
                csv_full_org in npi_full_org or
                npi_full_org in csv_full_org
            )
        )
        name_match = indiv_name_match or org_name_match
        csv_addr_norm = norm_addr(csv_addr)
        npi_addr_norm = norm_addr(npi_address)

        address_match = (
            csv_addr_norm != "" and npi_addr_norm != "" and (
                csv_addr_norm == npi_addr_norm or
                csv_addr_norm in npi_addr_norm or
                npi_addr_norm in csv_addr_norm
            )
        )
        return {
    "npi_valid": True,
    "npi_status": "ACTIVE",
    "npi_name": npi_name,
    "npi_address": npi_address,
    "npi_taxonomy_code": npi_taxonomy_code,
    "npi_taxonomy_desc": npi_taxonomy_desc,
    "name_match": name_match,
    "address_match": address_match,
}

    except Exception as e:
        return {
            "npi_valid": False,
            "npi_status": f"API_ERROR: {str(e)[:40]}",
        }


In [5]:
#defining the quality assurance agent
def quality_assurance_agent(row, validation_results, npi_results):
    score = 0.0
    # Structural checks
    if validation_results.get("npi_format_valid"):
        score += 0.1
    if validation_results.get("address_present"):
        score += 0.1
    # NPI validity
    if npi_results.get("npi_valid"):
        score += 0.4
    # Name & address consistency
    if npi_results.get("name_match"):
        score += 0.3
    if npi_results.get("address_match"):
        score += 0.1
    score = min(score, 1.0)
    if score >= 0.8:
        bucket = "AUTO_ACCEPT"
    elif score >= 0.5:
        bucket = "REVIEW"
    else:
        bucket = "REJECT"
    return {
        "confidence_score": round(score, 2),
        "qa_bucket": bucket,
        "name_match": bool(npi_results.get("name_match", False)),
        "address_match": bool(npi_results.get("address_match", False)),
    }



In [6]:
#Public Data Source Enrichment agent
def public_data_sources_enrichment_agent(row, npi_results):
    #Gemini is being used to guess extra info
    prompt = f"""
    Given NPI data: {npi_results}
    CSV data: {row.to_dict()}
    """
    try:
        resp = client.models.generate_content(model="gemini-1.5-flash", contents=prompt)
        import json, re
        json_str = re.search(r'\{.*\}', resp.text, re.S)
        if json_str:
            return json.loads(json_str.group(0))
    except:
        pass
    return {"provider_type": "UNKNOWN", "likely_specialty": "UNKNOWN", "risk_flags": []}


In [7]:
#defining the information enrichment agent
def information_enrichment_agent(row, enrichment_results, npi_results):
    taxonomy_desc = npi_results.get("npi_taxonomy_desc", "")
    llm_specialty = enrichment_results.get("likely_specialty", "UNKNOWN")
    final_specialty = llm_specialty if llm_specialty != "UNKNOWN" else taxonomy_desc
    return {
        "final_specialty": final_specialty,
        "provider_type": enrichment_results.get("provider_type", "UNKNOWN"),
        "enriched_flags": enrichment_results.get("risk_flags", []),
        "npi_taxonomy_desc": taxonomy_desc,
        "npi_taxonomy_code": npi_results.get("npi_taxonomy_code", "")
    }

In [8]:
#defining the directory management agent
def directory_management_agent(df_processed):
    auto = df_processed[df_processed["qa_bucket"] == "AUTO_ACCEPT"]
    review = df_processed[df_processed["qa_bucket"] == "REVIEW"]
    reject = df_processed[df_processed["qa_bucket"] == "REJECT"]
    print(f"\nDIRECTORY MANAGEMENT")
    print(f"AUTO_ACCEPT: {len(auto)} | REVIEW: {len(review)} | REJECT: {len(reject)}")
    auto.to_csv("directory_auto_accept.csv", index=False)
    review.to_csv("directory_review.csv", index=False)
    reject.to_csv("directory_reject.csv", index=False)
    return auto, review, reject

In [9]:
#definng the master orchestrator agent
def master_orchestrator_agent(df, limit=None):
    if limit:
        df_work = df.head(limit).copy()
    else:
        df_work = df.copy()
    list_containing_columns = ['risk_flags', 'enriched_flags']
    for col in list_containing_columns:
        if col not in df_work.columns:
            df_work[col] = pd.Series(index=df_work.index, dtype='object') # Initialize with NaN, but object dtype

    for idx, row in tqdm(df_work.iterrows(), total=len(df_work), desc="Processing"):
        # Agent 1: Data Validation
        v1 = data_validation_agent(row)
        # Agent 2: NPI Validation (API)
        v2 = npi_validation_agent(row)
        # Agent 3: Quality Assurance
        v3 = quality_assurance_agent(row, v1, v2)
        # Agent 4: Public Data Enrichment (LLM)
        v4 = public_data_sources_enrichment_agent(row, v2)
        # Agent 5: Information Enrichment
        v5 = information_enrichment_agent(row, v4,v2)

        # Store ALL results
        for agent_results in [v1, v2, v3, v4, v5]:
            for k, v in agent_results.items():
                df_work.at[idx, k] = v
        time.sleep(0.2)

    # Call directory_management_agent to get the auto, review, reject dataframes
    auto, review, reject = directory_management_agent(df_work)

    # Return df_work along with auto, review, reject to match the calling signature
    return df_work, auto, review, reject


In [10]:
print("Running the agent...")
df_final, auto, review, reject = master_orchestrator_agent(df, limit=None)
print("\n Pipeline complete! Check CSV outputs.")

Running the agent...


Processing: 100%|██████████| 1000/1000 [05:08<00:00,  3.24it/s]


DIRECTORY MANAGEMENT
AUTO_ACCEPT: 679 | REVIEW: 239 | REJECT: 82

 Pipeline complete! Check CSV outputs.


In [11]:
auto_existing   = pd.read_csv("directory_auto_accept.csv")
review_existing = pd.read_csv("directory_review.csv")
reject_existing = pd.read_csv("directory_reject.csv")


# This is the Sheet created by your Form (File → Share link)
sheet_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQPUXTUYda3P2Y2IZzcWD7hMvtU9JyBkGt5uL0OKiR-Enr37dXFRvJMnOyjwegLvb0pv2Kud7VpSIIC/pub?gid=256121927&single=true&output=csv"

# Convert to CSV export URL
csv_url = sheet_url.replace("/edit#gid=", "/export?format=csv&gid=")

df_raw = pd.read_csv(csv_url, on_bad_lines='skip')   # always gets latest responses


df_new_valid, auto_new, review_new, reject_new = master_orchestrator_agent(df_raw)

# Optional: tag source
auto_new["source"]   = "new"
review_new["source"] = "new"
reject_new["source"] = "new"

# === 4) APPEND new results to existing CSVs ===
auto_all   = pd.concat([auto_existing,   auto_new],   ignore_index=True)
review_all = pd.concat([review_existing, review_new], ignore_index=True)
reject_all = pd.concat([reject_existing, reject_new], ignore_index=True)

auto_all.to_csv("directory_auto_accept.csv", index=False)
review_all.to_csv("directory_review.csv", index=False)
reject_all.to_csv("directory_reject.csv", index=False)

Processing: 100%|██████████| 1/1 [00:00<00:00,  3.22it/s]


DIRECTORY MANAGEMENT
AUTO_ACCEPT: 0 | REVIEW: 0 | REJECT: 1


In [12]:
!pip install streamlit
!pip install ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 37.3 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd

st.set_page_config(page_title="Provider Validation Dashboard", layout="wide")
st.caption("BUILD v9 – using directory_auto/review/reject.csv ONLY")

@st.cache_data
def load_data():
    df_auto   = pd.read_csv("directory_auto_accept.csv")
    df_reject = pd.read_csv("directory_reject.csv")
    df_review = pd.read_csv("directory_review.csv")
    return df_auto, df_reject, df_review

df_auto, df_reject, df_review = load_data()
total_providers = len(df_auto) + len(df_reject) + len(df_review)

st.title("Provider Data Validation Dashboard")

col1, col2, col3, col4 = st.columns(4)
col1.metric("Total providers", total_providers)
col2.metric("Auto-accept", len(df_auto))
col3.metric("Review", len(df_review))
col4.metric("Reject", len(df_reject))

st.markdown("---")

view = st.radio(
    "Select view",
    ("AUTO_ACCEPT", "REJECT", "REVIEW"),
    horizontal=True
)

if view == "AUTO_ACCEPT":
    view_df = df_auto.copy()
elif view == "REJECT":
    view_df = df_reject.copy()
else:
    view_df = df_review.copy()

st.subheader(f"{view} providers")

st.sidebar.header("Filters")
min_conf, max_conf = st.sidebar.slider(
    "Confidence score range",
    0.0, 1.0,
    (0.0, 1.0),
    0.05
)

if "confidence_score" in view_df.columns:
    view_df = view_df[view_df["confidence_score"].between(min_conf, max_conf)]

cols_to_show = [c for c in [
    "npi",
    "provider_organization_name_(legal_business_name)",
    "provider_last_name_(legal_name)",
    "provider_first_name",
    "provider_first_line_business_practice_location_address",
    "qa_bucket",
    "confidence_score",
    "npi_status",
    "name_match",
    "address_match"
] if c in view_df.columns]

st.dataframe(view_df[cols_to_show].head(500), use_container_width=True)
st.markdown("---")
c1, c2 = st.columns(2)

with c1:
    if "confidence_score" in view_df.columns and not view_df.empty:
        st.markdown(f"**Confidence score distribution – {view}**")
        st.bar_chart(view_df["confidence_score"])

with c2:
    if "npi_status" in view_df.columns and not view_df.empty:
        st.markdown(f"**NPI status counts – {view}**")
        status_counts = view_df["npi_status"].value_counts().reset_index()
        status_counts.columns = ["npi_status", "count"]
        st.bar_chart(status_counts.set_index("npi_status"))


In [ ]:
!sed -n '1,40p' app.py


In [15]:
!kill -9 $(ps aux | grep 'streamlit run app.py' | awk '{print $2}') 2>/dev/null || echo "no old streamlit"
!kill -9 $(ps aux | grep ngrok | awk '{print $2}') 2>/dev/null || echo "no old ngrok"


^C
^C


In [17]:
!pip install pyngrok
from pyngrok import ngrok
from google.colab import userdata
import time # Import time for sleep

# Terminate any existing Streamlit and ngrok processes forcefully
!kill -9 $(ps aux | grep 'streamlit run app.py' | awk '{print $2}') 2>/dev/null || echo "no old streamlit"
!kill -9 $(ps aux | grep ngrok | awk '{print $2}') 2>/dev/null || echo "no old ngrok"

# Add a small delay to ensure processes are fully terminated
time.sleep(2)

NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Terminate any existing ngrok tunnels managed by pyngrok right before connecting
ngrok.kill()

public_url = ngrok.connect(8501)
print("NEW URL:", public_url)

!streamlit run app.py --server.port 8501 --server.headless true &>/dev/null &

^C
^C
NEW URL: NgrokTunnel: "https://agronomical-asha-undared.ngrok-free.dev" -> "http://localhost:8501"


In [18]:
import pandas as pd

raw = pd.read_csv("NPI_Extract_corr.csv")  # or NPI_Extract.csv
# Clean column names
raw.columns = [c.strip().lower().replace(" ", "_") for c in raw.columns]
print(raw[["npi",
           "provider_organization_name_(legal_business_name)",
           "provider_last_name_(legal_name)",
           "provider_first_name"]].head(10))


          npi provider_organization_name_(legal_business_name)  \
0  1679576722                                              NaN   
1  1588667638                                              NaN   
2  1497758544           CUMBERLAND COUNTY HOSPITAL SYSTEM, INC   
3  1306849450                                              NaN   
4  1215930367                                              NaN   
5  1023011178                                   COLLABRIA CARE   
6  1932102084                                              NaN   
7  1841293990                                              NaN   
8  1750384806                                              NaN   
9  1669475711                                              NaN   

  provider_last_name_(legal_name) provider_first_name  
0                           WIEBE               DAVID  
1                         PILCHER             WILLIAM  
2                             NaN                 NaN  
3                             NaN                